In [1]:
import autorootcwd

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from autogluon.tabular import TabularPredictor

In [3]:
from evaluation import search_knn, evaluate

In [4]:
# load files
df_test = pd.read_csv("homeworks/homework2/artificial_test.data", sep=' ', header=None)
df_train = pd.read_csv("homeworks/homework2/artificial_train.data", sep=' ', header=None)
df_train_labels = pd.read_csv("homeworks/homework2/artificial_train.labels", sep=' ', header=None)

# drop last column because its NaN for some reason
df_train = df_train.iloc[:, :-1]
df_test = df_test.iloc[:, :-1]

# convert df_train_labels to 0, 1
df_train_labels = df_train_labels.replace(-1, 0)

# convert to numpy
X_train = df_train.values
X_test = df_test.values
y_train = df_train_labels.values.ravel()

# show shapes
X_train.shape, X_test.shape, y_train.shape

((2000, 500), (600, 500), (2000,))

In [5]:
# convert back to df
df_train = pd.DataFrame(X_train)

# add target column
df_train['target'] = y_train

df_train

df_test = pd.DataFrame(X_test)
df_test

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,483,454,513,495,523,469,453,477,506,479,...,455,480,543,259,413,520,485,498,523,510
1,485,508,493,487,478,472,504,476,479,475,...,486,480,535,534,514,452,484,495,548,477
2,483,521,507,475,493,486,421,475,496,483,...,491,476,498,495,508,528,486,465,508,503
3,474,504,576,480,553,483,524,478,483,483,...,521,475,470,463,509,525,479,467,552,517
4,495,474,523,479,495,488,485,476,497,478,...,510,471,522,343,509,520,475,493,506,491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,493,458,503,478,517,479,472,478,444,477,...,469,475,485,443,517,486,474,489,506,506
596,481,484,481,490,449,481,467,478,469,483,...,506,485,508,599,498,527,481,490,455,451
597,485,485,530,480,444,487,462,475,509,494,...,442,474,502,368,453,482,478,481,484,517
598,477,469,528,485,483,469,482,477,494,476,...,473,476,453,638,471,538,470,490,613,492


In [16]:
# Train the model using AutoGluon with advanced configurations
predictor2 = TabularPredictor(label='target', eval_metric='balanced_accuracy').fit(
    train_data=df_train,
    presets='best_quality',
    num_stack_levels=3,
    num_bag_folds=4,
    time_limit=1200,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240116_213953"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=4, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 1200 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240116_213953/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.
Spend 303 seconds for the sub-fit(s) during dynamic stacking.
Time left for full fit of AutoGluon: 897 seconds.
Starting full fit now with num_stack_lev

In [7]:
# After training, you can get feature importance to understand which features are most impactful
# importance = predictor.feature_importance(df_train)

# Optional: Print feature importance
# print(importance)

In [22]:
# print ensembled models and their validaiton scores
# Get leaderboard information
leaderboard = predictor2.leaderboard(silent=True)

# Print details of ensembled models and their validation scores
print(leaderboard[['model', 'score_val']])

                          model  score_val
0            CatBoost_r9_BAG_L3     0.8670
1           WeightedEnsemble_L4     0.8670
2           WeightedEnsemble_L5     0.8670
3           WeightedEnsemble_L3     0.8665
4               LightGBM_BAG_L4     0.8665
..                          ...        ...
70    NeuralNetTorch_r79_BAG_L1     0.5965
71        NeuralNetTorch_BAG_L1     0.5855
72    NeuralNetTorch_r22_BAG_L1     0.5725
73       NeuralNetFastAI_BAG_L1     0.5635
74  NeuralNetFastAI_r191_BAG_L1     0.5420

[75 rows x 2 columns]


In [32]:
best_model = leaderboard.iloc[0]['model']  # Assuming the best model is at the top
val_score = leaderboard.iloc[0]['score_val']
best_model, val_score

('CatBoost_r9_BAG_L3', 0.867)

In [25]:
# generate predictions for test df
y_pred = predictor2.predict_proba(df_test)
y_pred.shape

(600, 2)

In [26]:
y_pred[1].shape

(600,)

In [27]:
np.savetxt("predictions_automl.txt", y_pred[1], fmt="%.2f", header="329532", comments="")

In [28]:

# load predictions
y_pred_2 = np.loadtxt("predictions_manual.txt")

# drop first row
y_pred_2 = y_pred_2[1:]
y_pred_2.shape

(600,)

In [30]:
# calculate balanced accuracy between y_pred and y_pred_2
from sklearn.metrics import balanced_accuracy_score
y_pred_class = np.where(np.array(y_pred[1]) > 0.5, 1, 0)

balanced_accuracy_score(y_pred_class, y_pred_2)

0.9317551847357817